In [2]:
import numpy

In [ ]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [ ]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [ ]:

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [ ]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

In [ ]:
import json
import nltk
import mysql.connector
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import random
from flask import Flask, render_template, request, jsonify  # Added jsonify
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the Flask app
app = Flask(__name__)
app.static_folder = 'static'

# Initialize NLTK
nltk.download('popular')
lemmatizer = WordNetLemmatizer()

# Load the pre-trained model and other data
model = load_model(r'C:\Users\gogul\OneDrive\Documents\Mental-health-Chatbot-master\Mental-health-Chatbot-master\model.h5')
intents = json.loads(open(r'Mental-health-Chatbot-master/intents.json').read())
words = pickle.load(open(r'C:\Users\gogul\OneDrive\Documents\Mental-health-Chatbot-master\Mental-health-Chatbot-master\texts.pkl', 'rb'))
classes = pickle.load(open(r'C:\Users\gogul\OneDrive\Documents\Mental-health-Chatbot-master\Mental-health-Chatbot-master\labels.pkl', 'rb'))

# Set up a MySQL database connection
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Gogul@2003",
    database="gogul"
)

cursor = db_connection.cursor()

# Create the chat_history table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS chat_history (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    user_input TEXT,
                    bot_response TEXT,
                    user_sentiment VARCHAR(255),  # Added column for user sentiment
                    bot_sentiment VARCHAR(255)    # Added column for bot sentiment
                )''')
db_connection.commit()

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into an array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create a short form for the word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if the current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by the strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def analyze_sentiment_vader(text):
    sentiment = analyzer.polarity_scores(text)
    compound_score = sentiment['compound']
    
    if compound_score >= 0.6:
        return "very_positive"
    
    elif 0.3 <= compound_score < 0.6:
        return "positive"
    elif 0.1 <= compound_score < 0.3:
        return "somewhat_positive"
    elif -0.1 <= compound_score < 0.1:
        return "neutral"
    elif -0.3 <= compound_score < -0.1:
        return "somewhat_negative"
    elif -0.6 <= compound_score < -0.3:
        return "negative"
    elif compound_score <= -0.6:
        return "very_negative"
    elif compound_score >= 0.5:
        return "joy"
    elif compound_score <= -0.5:
        return "sadness"
    elif compound_score >= 0.4:
        return "surprise"
    elif compound_score <= -0.4:
        return "disgust"
    elif compound_score >= 0.3:
        return "anticipation"
    elif compound_score <= -0.3:
        return "fear"
    elif compound_score >= 0.2:
        return "trust"
    elif compound_score <= -0.2:
        return "anger"
    elif compound_score >= 0.1:
        return "love"
    elif compound_score <= -0.1:
        return "hatred"
    elif compound_score >= 0.05:
        return "gratitude"
    elif compound_score <= -0.05:
        return "regret"
    else:
        return "casual"

def chatbot_response(msg):
    # Analyze sentiment of user input
    user_sentiment = analyze_sentiment_vader(msg)
    
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    
    # Analyze sentiment using VADER
    bot_sentiment = analyze_sentiment_vader(res)
    
    # Save the user input, bot's response, and sentiments to the database
    cursor.execute('INSERT INTO chat_history (user_input, bot_response, user_sentiment, bot_sentiment) VALUES (%s, %s, %s, %s)', (msg, res, user_sentiment, bot_sentiment))
    db_connection.commit()
    
    response = {
        "user_input": msg,
        "bot_response": res,
        "user_sentiment": user_sentiment,
        "bot_sentiment": bot_sentiment
    }
    
    return jsonify(response)

# Define a route for the home page
@app.route("/")
def home():
    return render_template("index.html")

# Define a route for handling user input and getting bot responses
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)

if __name__ == "__main__":
    app.run()


In [3]:
print("hi")

hi
